In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [9]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [10]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
data = np.array(data, dtype=np.int64)
r, c = data.shape

np.random.shuffle(data)

data_dev = data[0:1000].T
y_dev = data_dev[0]
x_dev = data_dev[1:c]


data_train = data[1000:r].T
y_train = data_train[0]
x_train = data_train[1:c]

x_train = x_train / 255.
_,m_train = x_train.shape

In [60]:
y_train

array([7, 7, 3, ..., 2, 4, 4])

In [61]:
def init_params():
    
    w1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    w2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    
    return w1, b1, w2, b2

def ReLu(z):    
    return np.maximum(z, 0)

def softmax(z):
    z = np.exp(z) / sum(np.exp(z))
    return z

def derive_ReLu(z):
    return z > 0

def forward_prop(w1, b1, w2, b2, x):
    
    z1 = w1.dot(x) + b1
    a1 = ReLu(z1)
    z2 = w2.dot(a1) + b2
    a2 = softmax(z2)
    
    return z1, a1, z2, a2

def one_hot_encode(y):
    
    one_hot_y = np.zeros((y.size, y.max() + 1))
    one_hot_y[np.arange(y.size), y] = 1
    one_hot_y = one_hot_y.T
    
    return one_hot_y
    
def back_prop(z1, a1, z2, a2, w1, w2, x, y):

    one_hot_y = one_hot_encode(y)
    
    dz2 = a2 - one_hot_y
    dw2 = 1 / r * dz2.dot(a1.T)
    db2 = 1 / r * np.sum(dz2)
    
    dz1 = w2.T.dot(dz2) * derive_ReLu(z1)
    dw1 = 1 / r * dz1.dot(x.T)
    db1 = 1 / r * np.sum(dz2)
    
    return dw1, db1, dw2, db2

def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    
    w1 = w1 - alpha * dw1
    b1 = b1 - alpha * db1
    
    w2 = w2 - alpha * dw2
    b2 = b2 - alpha * db2
    
    return w1, b1, w2, b2

In [62]:
def get_predictions(a2):
    return np.argmax(a2, 0)

def get_accuracy(predictions, y):
    print(predictions, y)
    return np.sum(predictions == y) / y.size

def gradient_desc(x, y, alpha, iterations):
    
    w1, b1, w2, b2 = init_params()
    
    for i in range(iterations):
        
        z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, x)
        dw1, db1, dw2, db2 = back_prop(z1, a1, z2, a2, w1, w2, x, y)
        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
        
        if i % 10 == 0 or i == iterations - 1:
            print('Iteration: ', i)
            print("Accuracy : ", get_accuracy(get_predictions(a2), y)) 
        
    return w1, b1, w2, b2

In [63]:
w1, b1, w2, b2 = gradient_desc(x_train, y_train, 0.10, 500)

Iteration:  0
[6 8 8 ... 2 6 8] [7 7 3 ... 2 4 4]
Accuracy :  0.09885365853658537
Iteration:  10
[4 9 3 ... 2 3 8] [7 7 3 ... 2 4 4]
Accuracy :  0.17673170731707316
Iteration:  20
[4 9 3 ... 2 3 9] [7 7 3 ... 2 4 4]
Accuracy :  0.2810487804878049
Iteration:  30
[4 6 3 ... 2 3 9] [7 7 3 ... 2 4 4]
Accuracy :  0.3728048780487805
Iteration:  40
[6 6 3 ... 7 3 7] [7 7 3 ... 2 4 4]
Accuracy :  0.43673170731707317
Iteration:  50
[9 6 3 ... 7 3 7] [7 7 3 ... 2 4 4]
Accuracy :  0.49702439024390244
Iteration:  60
[9 6 3 ... 7 2 6] [7 7 3 ... 2 4 4]
Accuracy :  0.5440731707317074
Iteration:  70
[9 6 3 ... 7 2 4] [7 7 3 ... 2 4 4]
Accuracy :  0.5834390243902439
Iteration:  80
[9 9 3 ... 7 2 4] [7 7 3 ... 2 4 4]
Accuracy :  0.6152682926829268
Iteration:  90
[9 7 3 ... 7 2 4] [7 7 3 ... 2 4 4]
Accuracy :  0.6390243902439025
Iteration:  100
[9 7 3 ... 7 2 4] [7 7 3 ... 2 4 4]
Accuracy :  0.6601219512195122
Iteration:  110
[9 7 3 ... 7 2 4] [7 7 3 ... 2 4 4]
Accuracy :  0.6767317073170732
Iteration: 

In [64]:
def make_predictions(x, w1, b1, w2, b2):
    
    _, _, _, a2 = forward_prop(w1, b1, w2, b2, x)
    predictions = get_predictions(a2)
    
    return predictions

def test_predictions( index, w1, b1, w2, b2):
    
    current_image = x_train[:, index, None]
    prediction = make_predictions(x_train[:, index, None], w1, b1, w2, b2)
    label = y_train[index]
    print('Prediction: ', prediction)
    print('True Label: ', label)
    

In [53]:
test_predictions(0, w1, b1, w2, b2)
test_predictions(55, w1, b1, w2, b2)
test_predictions(72, w1, b1, w2, b2)
test_predictions(908, w1, b1, w2, b2)

Prediction:  [4]
True Label:  4
Prediction:  [4]
True Label:  4
Prediction:  [8]
True Label:  8
Prediction:  [9]
True Label:  9


In [54]:
dev_predictions = make_predictions(x_dev, w1, b1, w2, b2)
get_accuracy(dev_predictions, y_dev)

[1 0 4 3 4 0 0 4 4 4 0 3 3 4 4 1 0 2 0 7 0 4 2 1 3 0 3 2 0 5 0 4 1 1 1 0 0
 0 2 0 0 4 4 0 4 6 0 3 0 5 0 4 1 4 7 7 1 1 0 1 4 7 1 7 4 4 2 3 4 4 0 0 0 7
 7 1 7 3 0 0 0 2 3 0 1 0 4 1 1 5 0 7 1 0 7 4 0 4 3 5 1 7 2 1 0 0 0 1 4 1 0
 1 2 7 0 0 0 1 4 1 5 0 0 0 2 0 4 0 1 0 1 0 0 0 1 7 1 5 0 0 4 4 1 0 1 1 0 1
 0 7 0 3 7 1 0 0 1 1 8 1 0 0 4 0 0 1 1 4 4 0 5 4 0 1 3 7 4 0 3 3 8 4 5 4 0
 4 7 8 0 1 5 2 1 1 0 0 7 3 2 0 1 0 4 0 1 4 0 7 1 1 4 2 0 2 1 0 0 1 4 4 1 0
 5 7 0 0 3 7 4 1 1 7 5 4 3 7 4 1 4 2 7 7 0 0 4 1 0 1 1 4 4 4 1 3 2 4 4 1 2
 0 2 0 0 7 2 1 0 5 2 1 1 5 0 1 8 0 1 0 0 1 4 0 1 4 1 0 4 2 1 4 1 1 1 0 8 0
 0 2 0 0 0 4 7 0 4 4 0 0 0 1 0 1 1 1 0 0 1 2 0 1 1 5 0 4 1 1 1 1 4 9 0 1 1
 8 0 0 3 0 5 8 4 0 1 5 4 5 0 1 2 0 0 4 0 0 0 0 4 3 3 5 1 4 4 3 1 4 2 0 2 1
 2 0 3 4 1 0 1 0 1 1 1 4 1 0 9 1 4 1 0 8 0 7 1 2 0 7 1 3 7 0 9 0 0 0 4 4 0
 0 0 3 4 1 0 2 7 4 0 4 1 0 1 2 0 0 4 0 0 0 1 4 0 1 3 0 0 1 3 4 0 0 2 0 1 0
 0 0 0 1 0 4 2 3 0 0 1 4 0 1 4 0 0 4 7 4 0 1 7 0 0 1 0 4 4 4 1 0 5 4 4 0 1
 0 4 0 4 1 3 1 0 5 0 1 1 

/tmp/ipykernel_42/1037690305.py:14: RuntimeWarning: overflow encountered in exp
  z = np.exp(z) / sum(np.exp(z))
/tmp/ipykernel_42/1037690305.py:14: RuntimeWarning: invalid value encountered in divide
  z = np.exp(z) / sum(np.exp(z))


0.422